##### https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

# LTSM

In [26]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sys

In [28]:
df = pd.read_csv('acc_data (right_wrist).csv')
X = np.column_stack((np.array(df['Acc_X']), np.array(df['Acc_Y']), np.array(df['Acc_Z'])))
y = np.array(df['PacketCounter']).reshape(-1, 1)

In [29]:
X.shape
y.shape

(403, 1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [31]:
X_train.shape
X_test.shape

(81, 3)

In [32]:
y_train.shape
y_test.shape

(81, 1)

In [33]:
# zero-offset class values
y_train = y_train - 1
y_test = y_test - 1

In [34]:
# one hot encode y
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(322, 3) (322, 401) (81, 3) (81, 402)


In [35]:
verbose, epochs, batch_size = 0, 15, 64
n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[1]
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

model.compile(optimizer=Adam(learning_rate = 0.01), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
#model.fit(X_train, y_train, epochs = 10, validation_data= (X_test, y_test), verbose=1)

In [1]:
history = model.fit(X_train, y_train, epochs = 10, validation_data= (X_test, y_test), verbose=1)

NameError: name 'model' is not defined

In [138]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath)
	return dataframe.values


In [139]:
# load a list of files and return as a 3d numpy array
def load_group(filenames):
	loaded = list()
	for name in filenames:
		data = load_file(name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [132]:
# load the dataset, returns train and test X and y elements
def load_dataset():
	# load all train
	trainX, trainy = load_group('Acc_X')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_group()
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [133]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [134]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [135]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [136]:
# run the experiment
run_experiment()

FileNotFoundError: [Errno 2] No such file or directory: 'A'

In [236]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sys
import scipy.stats as stats

In [238]:
df = pd.read_csv('acc_data (right_wrist).csv')
X = np.column_stack((np.array(df['Acc_X']), np.array(df['Acc_Y']), np.array(df['Acc_Z'])))
y = np.array(df['PacketCounter']).reshape(-1, 1)

In [239]:
columns =['PacketCounter', 'Acc_X', 'Acc_Y', 'Acc_Z']
data = pd.DataFrame(data = df, columns = columns)
data.head()

,PacketCounter,Acc_X,Acc_Y,Acc_Z
0,0,0.000000,0.000000,0.000000
1,1,-8.874124,-0.679627,2.768368
2,2,-8.873473,-0.613973,2.761706
3,3,-8.834931,-0.518206,2.797576
4,4,-8.921975,-0.508252,2.836315


In [240]:
data['Acc_X'] = data['Acc_X'].astype('float')
data['Acc_Y'] = data['Acc_Y'].astype('float')
data['Acc_Z'] = data['Acc_Z'].astype('float')

In [241]:
label = LabelEncoder()
data['label'] = label.fit_transform(data['PacketCounter'])
data.head()

,PacketCounter,Acc_X,Acc_Y,Acc_Z,label
0,0,0.000000,0.000000,0.000000,0
1,1,-8.874124,-0.679627,2.768368,1
2,2,-8.873473,-0.613973,2.761706,2
3,3,-8.834931,-0.518206,2.797576,3
4,4,-8.921975,-0.508252,2.836315,4


In [242]:
X = data[['Acc_X', 'Acc_Y', 'Acc_Z']]
y = data['label']

In [243]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

scaled_X = pd.DataFrame(data = X, columns = ['Acc_X', 'Acc_Y', 'Acc_Z'])
scaled_X['label'] = y.values

scaled_X.head()

,Acc_X,Acc_Y,Acc_Z,label
0,1.065724,-0.010709,-1.222835,0
1,-0.378791,-0.153964,-0.015481,1
2,-0.378685,-0.140125,-0.018386,2
3,-0.372411,-0.119939,-0.002742,3
4,-0.386580,-0.117841,0.014152,4


In [244]:
Fs = 1
frame_size = Fs*2 # 80
hop_size = Fs*1 # 40

In [245]:
def get_frames(df, frame_size, hop_size):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        x = df['Acc_X'].values[i: i + frame_size]
        y = df['Acc_Y'].values[i: i + frame_size]
        z = df['Acc_Z'].values[i: i + frame_size]
        
        # Retrieve the most often used label in this segment
        label = stats.mode(df['label'][i: i + frame_size])[0][0]
        frames.append([x, y, z])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

X, y = get_frames(scaled_X, frame_size, hop_size)

X.shape, y.shape

((401, 2, 3), (401,))

In [246]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [247]:
X_train.shape, X_test.shape

((320, 2, 3), (81, 2, 3))

In [248]:
X_train[0].shape, X_test[0].shape

((2, 3), (2, 3))

In [249]:
X_train = X_train.reshape(320, 3, -1)
X_test = X_test.reshape(81, 3, -1)

In [250]:
X_train[0].shape, X_test[0].shape

((3, 2), (3, 2))

In [251]:
verbose, epochs, batch_size = 0, 10, 128
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[1], y_train.shape[0]
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [265]:
# fit and evaluate a model
def evaluate_model(X_train, y_trainy, X_test, y_test):
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[1], y_train.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
	return accuracy

In [266]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [267]:
scores = list()
score = evaluate_model(X_train, y_train, X_test, y_test)
score = score * 100.0
print('>#%d: %.3f' % (r+1, score))
scores.append(score)
# summarize results
summarize_results(scores)

IndexError: tuple index out of range

In [268]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(X_train, y_train, X_test, y_test)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)


In [269]:
# run the experiment
run_experiment()

IndexError: tuple index out of range

In [252]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 100)               41600     
_________________________________________________________________
dropout_23 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_46 (Dense)             (None, 320)               32320     
Total params: 84,020
Trainable params: 84,020
Non-trainable params: 0
_________________________________________________________________


In [255]:
#batch_size = 128
#epochs = 10
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)


ValueError: Error when checking input: expected lstm_23_input to have shape (3, 3) but got array with shape (3, 2)